# COVID-19 Dashboard

## Load Libraries

In [1]:
import pandas as pd
import numpy as np
import io
import requests
import plotly.graph_objects as go
import plotly.express as px

## Load Data

In [2]:
urls = "https://covid19.isciii.es/resources/serie_historica_acumulados.csv"
sp = requests.get(urls).content
dfs = pd.read_csv(io.StringIO(sp.decode('latin1')))

df_loc = pd.DataFrame([
    ('AN','Andalusia',-4.5000000, 37.6000000),
    ('AR','Aragon',-1.0000000, 41.0000000),
    ('AS','Asturias', -6.0000000, 43.3333333),
    ('IB','Balearic Islands', 2.4334717, 39.2226789),
    ('PV','Basque Country', -2.7500000, 43.0000000),
    ('CN','Canary Islands', -15.5000000, 28.0000000),
    ('CB','Cantabria', -4.0000000, 43.3333333),
    ('CL','Castille and Leon', -4.2500000, 41.6666667),
    ('CM','Castille-La Mancha', -3.0000000, 39.5000000),
    ('CT','Catalonia', 1.8676758, 41.8204551),
    ('EX','Extremadura', -6.0000000, 39.0000000),
    ('GA','Galicia', -7.8662109, 42.7550796),
    ('RI','La Rioja', -2.5000000, 42.2500000),
    ('MD','Madrid', -3.6906338, 40.4252581),
    ('MC','Murcia', -1.5000000, 38.0000000),
    ('NC','Navarre', -1.6666667, 42.7500000),
    ('VC','Valencia', -0.7500000, 39.5000000)],
   columns=['region','region_name','long','lat'])

In [3]:
dfs = dfs.iloc[:-2,:]
dfs.columns = ['region','date','cases','hospitalised','ICU','deaths','recovered']
dfs.date = pd.to_datetime(dfs.date, format='%d/%m/%Y')
dfs = dfs.sort_values(['region','date'])

In [4]:
dfs = pd.merge(dfs, df_loc[['region','region_name']], how='inner', on='region')
dfs.head()

,region,date,cases,hospitalised,ICU,deaths,recovered,region_name
0,AN,2020-02-20,NaN,NaN,NaN,NaN,NaN,Andalusia
1,AN,2020-02-21,NaN,NaN,NaN,NaN,NaN,Andalusia
2,AN,2020-02-22,NaN,NaN,NaN,NaN,NaN,Andalusia
3,AN,2020-02-23,NaN,NaN,NaN,NaN,NaN,Andalusia
4,AN,2020-02-24,NaN,NaN,NaN,NaN,NaN,Andalusia


In [5]:
###############################################################################
#### Line Graph

coloursp = ['#636EFA', '#EF553B', '#00CC96', '#AB63FA', '#FFA15A', '#19D3F3',
'#FF6692', '#B6E880', '#FF97FF', '#FECB52', '#636EFA', '#EF553B', '#00CC96',
'#AB63FA', '#FFA15A', '#19D3F3', '#FF6692', '#B6E880', '#FF97FF']

color_balck = 'rgb(67,67,67)'


In [6]:
###############################################################################
### Confirmed

dfs_pivot_c= dfs.pivot(index='date', columns='region_name',values='cases')

dfs_norm_10case = []
for column in dfs_pivot_c.columns:
    dfs_norm_10case.append(dfs_pivot_c[column][dfs_pivot_c[column] >= 10].values)

dfs_norm_10case = pd.DataFrame(dfs_norm_10case).T

dfs_norm_10case.columns = dfs_pivot_c.columns

dfs_norm_10case = dfs_norm_10case.dropna(how='all',axis = 1)

y_cutoff = dfs_norm_10case.max().max() + dfs_norm_10case.max().max() * 0.2
x_start = 10
every_1day = [x_start]
every_2days = [x_start]
every_3days = [x_start]
every_4days = [x_start]
every_week = [x_start]
for i in range(1,dfs_norm_10case.shape[0] + 10):
    x = x_start * (2 ** i)
    if x > y_cutoff:
        x = None
        every_1day.append(x)
    else: every_1day.append(x)
    y = x_start * ((2 ** (1/2)) ** i)
    if y > y_cutoff:
        y = None
        every_2days.append(y)
    else: every_2days.append(y)
    z = x_start * ((2**(1/3)) ** i)
    if z > y_cutoff:
        z = None
        every_3days.append(z)
    else: every_3days.append(z)
    v = x_start * ((2**(1/4)) ** i)
    if v > y_cutoff:
        v = None
        every_4days.append(v)
    else: every_4days.append(v)
    w = x_start * ((2**(1/7)) ** i)
    if w > y_cutoff:
        w = None
        every_week.append(w)
    else: every_week.append(w)

Lines_cs = pd.DataFrame(every_1day,columns=['every 1 day'],dtype=np.float64)
Lines_cs['every 2 days'] = pd.DataFrame(every_2days)
Lines_cs['every 3 days'] = pd.DataFrame(every_3days)
Lines_cs['every 4 days'] = pd.DataFrame(every_4days)
Lines_cs['every week'] = pd.DataFrame(every_week)

In [146]:
###########################################################################
#### Confirmed

data = dfs_norm_10case
lines = Lines_cs
title = 'Comulative number of confirmed cases, by number of days since 10th case'
x_title = 'Number of days since 10th case'
y_title = 'Comulative number of cases'
yaxis_scale_value = 'log'

fig = go.Figure()

plot_data = []
annotations = []
for i, column in enumerate(data.columns):
    plot_data.append(
        go.Scatter(x=list(data.index.values),
                   y=data[column],
                   hovertemplate = '%{y:.0f}',
                   mode='lines',
                   line = {'color': coloursp[i]},
                   opacity=0.6,
                   name= column ))

    plot_data.append(
        go.Scatter(x = [data[column].dropna().index[-1]],
                   y = [list(data[column].dropna())[-1]],
                   mode='markers',
                   marker = {'color': coloursp[i],
                             'size' : 5},
                   showlegend=False,
                   opacity=0.6,
                   hoverinfo='skip',) )

    annotations.append(dict(xref='x',yref='y',
                            x=data[column].dropna().index[-1],
                            y=list(data[column].dropna())[-1],
                            xanchor='left', yanchor='middle',
                            text=df_loc.set_index('region_name').region.loc[column],
                            font={'family':'Arial','size':12},
                            showarrow=False))
for column in lines.columns:
    plot_data.append(
        go.Scatter(x=list(lines.index.values),
                   y=lines[column],
                   hovertemplate = 'Doubles',
                   mode='lines',
                   opacity=0.5,
                   line={'color': color_balck,
                         'dash':'dash'},
                   name=column) )
    
for i in range(0,plot_data.__len__()):
    fig.add_trace(plot_data[i])
    
if yaxis_scale_value == 'log':
    for i in annotations:
        i['y'] = np.log10(i['y'])
elif yaxis_scale_value == 'linear':
    annotations = annotations

fig.update_layout(  title = title,
                    width = 900,
                    height = 500,
                    xaxis={'title': x_title,},
                    yaxis={'title': y_title,
                           'type': 'log' if yaxis_scale_value == 'log' else 'linear'},
                    hovermode='x',
                    annotations = annotations,
                    template = 'plotly_white' 
                 )
    
fig.show()

In [8]:
###############################################################################
### Deaths

dfs_pivot_d = dfs.pivot(index='date', columns='region_name',values='deaths')

dfs_norm_3death = []
for column in dfs_pivot_d.columns:
    dfs_norm_3death.append(dfs_pivot_d[column][dfs_pivot_d[column] >= 3].values)

dfs_norm_3death = pd.DataFrame(dfs_norm_3death).T

dfs_norm_3death.columns = dfs_pivot_d.columns

dfs_norm_3death = dfs_norm_3death.dropna(how='all',axis = 1)

y_cutoff = 6000
x_start = 3
every_1day = [x_start]
every_2days = [x_start]
every_3days = [x_start]
every_week = [x_start]
for i in range(1,dfs_norm_3death.shape[0]+10):
    x = x_start * (2 ** i)
    if x > y_cutoff:
        x = None
        every_1day.append(x)
    else: every_1day.append(x)
    y = x_start * ((2 ** (1/2)) ** i)
    if y > y_cutoff:
        y = None
        every_2days.append(y)
    else: every_2days.append(y)
    z = x_start * ((2**(1/3)) ** i)
    if z > y_cutoff:
        z = None
        every_3days.append(z)
    else: every_3days.append(z)
    w = x_start * ((2**(1/7)) ** i)
    if w > y_cutoff:
        w = None
        every_week.append(w)
    else: every_week.append(w)

Lines_ds = pd.DataFrame(every_1day,columns=['every 1 day'],dtype=np.float64)
Lines_ds['every 2 days'] = pd.DataFrame(every_2days)
Lines_ds['every 3 days'] = pd.DataFrame(every_3days)
Lines_ds['every week'] = pd.DataFrame(every_week)

In [9]:
###########################################################################
#### Deaths

data = dfs_norm_3death
lines = Lines_ds
title = 'Comulative number of deaths, by number of days since 3rd death'
x_title = 'Number of days since 3rd death'
y_title = 'Comulative number of deaths'
yaxis_scale_value = 'log'

fig = go.Figure()

plot_data = []
annotations = []
for i, column in enumerate(data.columns):
    plot_data.append(
        go.Scatter(x=list(data.index.values),
                   y=data[column],
                   hovertemplate = '%{y:.0f}',
                   mode='lines',
                   line = {'color': coloursp[i]},
                   opacity=0.6,
                   name=column) )

    plot_data.append(
        go.Scatter(x = [data[column].dropna().index[-1]],
                   y = [list(data[column].dropna())[-1]],
                   mode='markers',
                   marker = {'color': coloursp[i],
                             'size' : 5},
                   showlegend=False,
                   opacity=0.6,
                   hoverinfo='skip',) )

    annotations.append(dict(xref='x',yref='y',
                            x=data[column].dropna().index[-1],
                            y=list(data[column].dropna())[-1],
                            xanchor='left', yanchor='middle',
                            text=df_loc.set_index('region_name').region.loc[column],
                            font={'family':'Arial','size':12},
                            showarrow=False))
for column in lines.columns:
    plot_data.append(
        go.Scatter(x=list(lines.index.values),
                   y=lines[column],
                   hovertemplate = 'Doubles',
                   mode='lines',
                   opacity=0.5,
                   line={'color': color_balck,
                         'dash':'dash'},
                   name=column) )
    
for i in range(0,plot_data.__len__()):
    fig.add_trace(plot_data[i])
    
if yaxis_scale_value == 'log':
    for i in annotations:
        i['y'] = np.log10(i['y'])
elif yaxis_scale_value == 'linear':
    annotations = annotations

fig.update_layout(  title = title,
                    width = 900,
                    height = 500,
                    xaxis={'title': x_title,},
                    yaxis={'title': y_title,
                           'type': 'log' if yaxis_scale_value == 'log' else 'linear'},
                    hovermode='x',
                    annotations = annotations
                 )
    
fig.show()

In [10]:
###############################################################################
### Deaths - Daily

dfs_pivot_d_daily = dfs_pivot_d.diff()
dfs_pivot_d_daily.loc['2020-03-08'] = dfs_pivot_d.loc['2020-03-08']

regions = dfs_pivot_d_daily.agg(max) > 3
regions = (regions[regions == True].index.values)
dfs_pivot_3death = dfs_pivot_d_daily[regions]

dfs_norm_3death_roll = []
for column in dfs_pivot_3death.columns:
    temp = dfs_pivot_3death[column].reset_index(drop=True)
    dfs_norm_3death_roll.append(dfs_pivot_3death[column].iloc[temp[temp > 3]\
                                                             .index[0]:].values)

dfs_norm_3death_roll = pd.DataFrame(dfs_norm_3death_roll).T
dfs_norm_3death_roll.columns = dfs_pivot_3death.columns
dfs_norm_3death_roll = dfs_norm_3death_roll.rolling(7).mean().iloc[6:]\
                                                         .reset_index(drop=True)
dfs_norm_3death_roll = dfs_norm_3death_roll.dropna(how='all',axis = 1)

In [11]:
###########################################################################
#### Daily Deaths

data = dfs_norm_3death_roll
title = 'Daily deaths'
x_title = 'Number of days since 3 daily deaths first recorded'
y_title = 'Number of daily deaths (7 day rolling average)'
yaxis_scale_value = 'linear'

fig = go.Figure()

plot_data = []
annotations = []
for i, column in enumerate(data.columns):
    plot_data.append(
        go.Scatter(x=list(data.index.values),
                   y=data[column],
                   hovertemplate = '%{y:.0f}',
                   mode='lines',
                   line = {'color': coloursp[i]},
                   opacity=0.6,
                   name=column) )

    plot_data.append(
        go.Scatter(x = [data[column].dropna().index[-1]],
                   y = [list(data[column].dropna())[-1]],
                   mode='markers',
                   marker = {'color': coloursp[i],
                             'size' : 5},
                   showlegend=False,
                   opacity=0.6,
                   hoverinfo='skip',) )

    annotations.append(dict(xref='x',yref='y',
                            x=data[column].dropna().index[-1],
                            y=list(data[column].dropna())[-1],
                            xanchor='left', yanchor='middle',
                            text=df_loc.set_index('region_name').region.loc[column],
                            font={'family':'Arial','size':12},
                            showarrow=False))
    
for i in range(0,plot_data.__len__()):
    fig.add_trace(plot_data[i])
    
if yaxis_scale_value == 'log':
    for i in annotations:
        i['y'] = np.log10(i['y'])
elif yaxis_scale_value == 'linear':
    annotations = annotations

fig.update_layout(  title = title,
                    width = 900,
                    height = 500,
                    xaxis={'title': x_title,},
                    yaxis={'title': y_title,
                           'type': 'log' if yaxis_scale_value == 'log' else 'linear'},
                    hovermode='x',
                    annotations = annotations
                 )
    
fig.show()

In [143]:
###############################################################################
### Geo-plot

df_geos = pd.merge(dfs, df_loc.drop('region_name',axis=1), how='inner', on='region').fillna(0)
datas = df_geos[df_geos.date == df_geos.date.unique()[-1]]

In [145]:
fig_spain_map = go.Figure(go.Scattergeo(
                                        lon = datas['long'],
                                        lat = datas['lat'],
                                        text = datas['region_name']+': '+datas['cases'].astype(str),
                                        mode = 'markers',
                                        marker = dict(size = datas['cases']/40,
                                                    color = coloursp,
                                                    line_width=0.5,
                                                    sizemode = 'area'),
                                        )
                           )

fig_spain_map.add_trace(go.Scattergeo(
                                        lon = datas['long'],
                                        lat = datas['lat'],
                                        text = datas['region'],
                                        mode = 'text',
                                        marker = dict(size = 0,
                                                    color = coloursp,
                                                    line_width=0.0,
                                                    sizemode = 'area'),
                                        ))

fig_spain_map.update_layout(  title = 'Confirmed cases in Spain by region',
                    width = 900,
                    height = 500,
                    showlegend=False
                 )

fig_spain_map.update_geos(fitbounds="locations")

fig_spain_map.show()

In [17]:
data_sun = datas[['region','cases','hospitalised','ICU','deaths','recovered']].copy()
data_sun['active'] = pd.Series(data_sun.loc[:,'cases'] -\
                              ( data_sun.loc[:,'deaths'] + data_sun.loc[:,'recovered']))
data_sun['home_isolation'] = pd.Series(data_sun.loc[:,'active']\
                                       - ( data_sun.loc[:,'hospitalised']))
data_sun = data_sun.reset_index(drop=True)
data_sun['home_isolation'][7] = data_sun.loc[:,'home_isolation'][7] * (-1)
data_sun['home_isolation'][14] = data_sun.loc[:,'home_isolation'][14] * (-1)

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [112]:
data_sun_total = data_sun[['region','cases']]
data_sun_total = pd.melt(data_sun_total, id_vars=['region'], value_vars=['cases'])

data_sun_cases = pd.melt(data_sun, id_vars=['region'],value_vars=['active','deaths','recovered'])
data_sun_cases = data_sun_cases.sort_values('region')

data_sun_active = pd.melt(data_sun, id_vars=['active'],value_vars=['hospitalised','ICU','home_isolation'])
data_sun_active['active'] = data_sun_active['active'].apply(str) 
data_sun_active = data_sun_active.sort_values('active')

for r in df_loc.region:
    data_sun_cases[data_sun_cases.region == r] = \
    data_sun_cases[data_sun_cases.region == r].replace('active','active ('+r+')')

a = [ 'active ('+ i +')' for i in list(df_loc.region)] * 3
a.sort()

In [113]:
labels = list(data_sun_total.region) + list(data_sun_cases.variable) + list(data_sun_active.variable)
parents = ['Spain'] * len(data_sun_total.region) \
        + list(data_sun_cases['region']) + a
values= list(data_sun_total['value']) + list(data_sun_cases['value'])+ list(data_sun_active['value'])

In [137]:
fig_pie_chart =go.Figure(go.Sunburst(labels = labels, parents = parents, values= values,
                                     #branchvalues="total",
                                     outsidetextfont = {"size": 20, "color": "#377eb8"},
                                     insidetextfont = {"size": 15},
                                    ))

fig_pie_chart.update_layout(width = 1000,height = 1000,
                            margin = dict(t=0, l=0, r=0, b=0))

fig_pie_chart.show()